# Code Generator: Python to Go

Use a frontier model to generate high-performance **Go** code from Python code.

**Note:** You can run the generated Go code locally (after installing the Go toolchain) or use an online compiler such as [Go Playground](https://go.dev/play/).

# Imports

In [1]:
import os
import io
import sys
import json
import shutil
import subprocess
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from IPython.display import Markdown, display

# Allow importing from parent (week4) when running from community-contributions
_week4_root = os.path.dirname(os.getcwd()) if os.path.basename(os.getcwd()) == "community-contributions" else os.getcwd()
sys.path.insert(0, _week4_root)
from system_info import retrieve_system_info

# Load API keys

In [2]:
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")
grok_api_key = os.getenv("GROK_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (optional)")
if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (optional)")
if grok_api_key:
    print(f"Grok API Key exists and begins {grok_api_key[:4]}")
else:
    print("Grok API Key not set (optional)")
if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (optional)")
if openrouter_api_key:
    print(f"OpenRouter API Key exists and begins {openrouter_api_key[:6]}")
else:
    print("OpenRouter API Key not set (optional)")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key not set (optional)
Google API Key not set (optional)
Grok API Key not set (optional)
Groq API Key not set (optional)
OpenRouter API Key exists and begins sk-or-


# Connect to OpenRouter

All models are called via the **OpenRouter API** (one API key, one base URL). Set `OPENROUTER_API_KEY` in your environment.

In [3]:
openrouter_client = (
    OpenAI(api_key=openrouter_api_key, base_url="https://openrouter.ai/api/v1")
    if openrouter_api_key
    else None
)

models = [
    "openai/gpt-4o",
    "openai/gpt-4o-mini",
    "anthropic/claude-3.5-sonnet",
    "anthropic/claude-3-haiku",
    "google/gemini-2.0-flash-exp:free",
    "google/gemini-flash-1.5",
    "meta-llama/llama-3.1-70b-instruct",
    "mistralai/mistral-large",
    "deepseek/deepseek-coder",
    "qwen/qwen-2.5-coder-32b-instruct",
]
clients = {m: openrouter_client for m in models}
models = [m for m in models if openrouter_client]
if not models:
    print("Set OPENROUTER_API_KEY in your environment to use models.")

# System info and Go toolchain

In [7]:
def go_toolchain_info():
    """Return Go toolchain presence and version for LLM context."""
    go_path = shutil.which("go")
    info = {
        "installed": bool(go_path),
        "go_path": go_path or "",
        "version": "",
        "GOROOT": os.environ.get("GOROOT", ""),
        "GOPATH": os.environ.get("GOPATH", ""),
    }
    if go_path:
        try:
            out = subprocess.check_output(
                [go_path, "version"], text=True, stderr=subprocess.DEVNULL, timeout=3
            )
            info["version"] = out.strip().split("\n")[0]
        except Exception:
            pass
    return info

system_info = retrieve_system_info()
go_info = go_toolchain_info()
print("Go installed:", go_info["installed"])
if go_info["version"]:
    print("Go version:", go_info["version"])

Go installed: True
Go version: go version go1.26.0 darwin/arm64


## Optional: Ask the model for Go build/run commands

You can run this once to get suggested `compile_command` and `run_command` for your system.

In [8]:
message = f"""
Here is a report of the system information for my computer.
I want to compile a single Go file called main.go and then execute the binary in the simplest way possible.
Please reply with whether I need to install the Go toolchain. If so, give the simplest step-by-step instructions.

If Go is already installed, tell me exactly what to use in Python for:
compile_command = [...]  # e.g. ["go", "build", "-ldflags=-s -w", "-o", "main", "main.go"]
run_command = [...]       # e.g. ["./main"] on Unix or [".\\main.exe"] on Windows

Prioritize fastest possible runtime performance (use -ldflags="-s -w" and appropriate build flags).
Reply with the commands in markdown.

System information:
{json.dumps(system_info, indent=2)}

Go toolchain:
{json.dumps(go_info, indent=2)}
"""

if openrouter_client and models:
    response = openrouter_client.chat.completions.create(
        model=models[0], messages=[{"role": "user", "content": message}]
    )
    display(Markdown(response.choices[0].message.content))
else:
    print("Set OPENROUTER_API_KEY and re-run the API key + client cells first.")

As the Go toolchain is already installed on your system at `/opt/homebrew/bin/go` and you are using an `arm64` macOS machine, you can compile and run your Go program with the following steps to ensure the fastest runtime performance:

### Compile Command

Use the following command to compile your `main.go` file:

```python
compile_command = ["go", "build", "-ldflags=-s -w", "-o", "main", "main.go"]
```

### Run Command

Once compiled, run the binary with:

```python
run_command = ["./main"]
```

These commands should be executed in a terminal or a script, ensuring that your Go environment is correctly set up. The `-ldflags=-s -w` flags remove symbol table and debug information from the binary, reducing its size and potentially improving load times.

## Go compile and run commands

Adjust for your OS (e.g. use `main.exe` and `.\\main.exe` on Windows).

In [9]:
compile_command = ["go", "build", "-ldflags=-s -w", "-o", "main", "main.go"]
run_command = ["./main"]  # Windows: [".\\main.exe"]

# Main task: Port Python to Go

In [10]:
language = "Go"
extension = "go"

system_prompt = f"""
Your task is to convert Python code into high-performance {language} code.
Respond only with {language} code. Do not provide any explanation other than occasional comments.
The {language} code must produce identical output to the Python code and run as fast as possible.
Use standard library only unless the algorithm clearly benefits from a well-known package (e.g. math).
Output must be a complete, runnable program (e.g. package main with func main()).

Important for correctness: If the Python code uses 2**32 (or any value that does not fit in 32 bits),
use uint64 for that value and for any LCG/random state. In Go, uint32 max is 2^32-1, so the literal
4294967296 (2^32) overflows uint32. Use uint64 and e.g. m := uint64(1)<<32 or var m uint64 = 1<<32.
"""

def user_prompt_for(python_code):
    return f"""
Port this Python code to {language} with the fastest possible implementation that produces identical output.

System information:
{json.dumps(system_info, indent=2)}

Go toolchain:
{json.dumps(go_info, indent=2)}

Your response will be written to main.{extension} and then compiled and run with:
Compile: {compile_command}
Run: {run_command}

Respond only with {language} code. No markdown fences or explanation.

Python code to port:

```python
{python_code}
```
"""

In [11]:
def messages_for(python_code):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(python_code)},
    ]

In [12]:
def write_output(code):
    with open(f"main.{extension}", "w") as f:
        f.write(code)

In [14]:
def port(model, python_code):
    client = clients.get(model)
    if not client:
        return f"Error: No client for model {model}"
    kwargs = {"model": model, "messages": messages_for(python_code), "max_tokens": 4096}
    if "gpt" in model.lower():
        try:
            kwargs["reasoning_effort"] = "high"
        except Exception:
            pass
    response = client.chat.completions.create(**kwargs)
    reply = response.choices[0].message.content or ""
    for strip in ("```go", "```Go", "```"):
        reply = reply.replace(strip, "")
    return reply.strip()

In [15]:
def run_python(code):
    globals_dict = {"__builtins__": __builtins__}
    buffer = io.StringIO()
    old_stdout = sys.stdout
    sys.stdout = buffer
    try:
        exec(code, globals_dict)
        output = buffer.getvalue()
    except Exception as e:
        output = f"Error: {e}"
    finally:
        sys.stdout = old_stdout
    return output

In [16]:
def compile_and_run(go_code):
    write_output(go_code)
    try:
        subprocess.run(compile_command, check=True, text=True, capture_output=True, cwd=os.getcwd())
        run_result = subprocess.run(run_command, check=True, text=True, capture_output=True, cwd=os.getcwd())
        return run_result.stdout
    except subprocess.CalledProcessError as e:
        return f"An error occurred:\n{(e.stderr or e.stdout or str(e))}"

# Example Python code (max subarray sum with LCG)

In [17]:
python_hard = """# Be careful to support large numbers

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

# Gradio UI

In [19]:
try:
    from styles import CSS
except ImportError:
    CSS = None

with gr.Blocks(css=CSS or "", theme=gr.themes.Monochrome(), title=f"Port from Python to {language}") as ui:
    with gr.Row(equal_height=True):
        with gr.Column(scale=6):
            python_input = gr.Code(
                label="Python (original)",
                value=python_hard,
                language="python",
                lines=26,
            )
        with gr.Column(scale=6):
            go_output = gr.Code(
                label=f"{language} (generated)",
                value="",
                language="markdown",
                lines=26,
            )

    with gr.Row(elem_classes=["controls"]):
        python_run_btn = gr.Button("Run Python", elem_classes=["run-btn", "py"])
        model_dropdown = gr.Dropdown(models, value=models[0] if models else None, show_label=False)
        convert_btn = gr.Button(f"Port to {language}", elem_classes=["convert-btn"])
        go_run_btn = gr.Button(f"Run {language}", elem_classes=["run-btn", "cpp"])

    with gr.Row(equal_height=True):
        with gr.Column(scale=6):
            python_out = gr.TextArea(label="Python result", lines=8, elem_classes=["py-out"])
        with gr.Column(scale=6):
            go_out = gr.TextArea(label=f"{language} result", lines=8, elem_classes=["cpp-out"])

    convert_btn.click(fn=port, inputs=[model_dropdown, python_input], outputs=[go_output])
    python_run_btn.click(fn=run_python, inputs=[python_input], outputs=[python_out])
    go_run_btn.click(fn=compile_and_run, inputs=[go_output], outputs=[go_out])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7873
* To create a public link, set `share=True` in `launch()`.
